<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/SMA_Short_VIX_Adjust.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##SMA_Short_VIX_Adjust
---
This notebook analyzes the historical price data of the ticker to evaluate the impact of **Simple Moving Average (SMA)** sign changes on asset returns.

* It begins by downloading historical data via the `yfinance` library and cleaning the resulting DataFrame into a flat table format.
* A custom function calculates the SMA for a given window and identifies "sign changes," which occur when the closing price crosses above or below the moving average.
* The analysis specifically filters for instances where the price drops below the SMA (sign change to -1) and calculates the subsequent return between those events.
* The script iterates through a range of lookback periods (from 3 to 21) to aggregate these returns by year, allowing for a comparison of how different SMA lengths perform over time.
* The final output is a concatenated DataFrame that summarizes the total yearly returns triggered by these SMA crossovers for each lookback value.

---

In [28]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

# Define a ticker and a date range for your data
ticker = 'UVXY'
start_date = '2012-01-01'
end_date = '2025-12-25'
window = 5
lookback_range = [3,21]
vix_limit = 0

# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
data = data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
data = data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
data.insert(1, 'Ticker', ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,UVXY,1.632000e+11,1.675500e+11,1.612500e+11,1.656000e+11,0
1,2012-01-04,UVXY,1.569000e+11,1.705500e+11,1.564500e+11,1.678500e+11,0
2,2012-01-05,UVXY,1.497000e+11,1.642500e+11,1.492500e+11,1.611000e+11,0
3,2012-01-06,UVXY,1.449000e+11,1.530000e+11,1.419000e+11,1.471500e+11,0
4,2012-01-09,UVXY,1.405500e+11,1.446000e+11,1.390500e+11,1.422000e+11,0
...,...,...,...,...,...,...,...
3511,2025-12-18,UVXY,4.108000e+01,4.255000e+01,4.060000e+01,4.165000e+01,10864500
3512,2025-12-19,UVXY,3.866000e+01,4.045000e+01,3.853000e+01,4.040000e+01,6814300
3513,2025-12-22,UVXY,3.680000e+01,3.784000e+01,3.667000e+01,3.754000e+01,5383700
3514,2025-12-23,UVXY,3.698000e+01,3.716000e+01,3.661000e+01,3.708000e+01,4483100


In [29]:
# Download historical data from Yahoo Finance for a single ticker.
# This will result in a DataFrame with 'Date' as a simple index.
vix_ticker = '^VIX'
vix_data = yf.download(vix_ticker, start=start_date, end=end_date, auto_adjust=True)

# Use reset_index() to convert the 'Date' index into a column.
vix_data = vix_data.reset_index()

# Now, to get a new DataFrame with just the 'Price' level, we can use droplevel()
# This removes the 'Ticker' level from the columns, leaving only the 'Price' level.
vix_data = vix_data.droplevel(level='Ticker', axis=1)

# Now, add the 'Ticker' column at position 1 (right after the 'Date' column).
vix_data.insert(1, 'Ticker', vix_ticker)

# The DataFrame is now a flat table with no MultiIndex.
display(vix_data)

[*********************100%***********************]  1 of 1 completed


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2012-01-03,^VIX,22.969999,23.100000,22.540001,22.950001,0
1,2012-01-04,^VIX,22.219999,23.730000,22.219999,23.440001,0
2,2012-01-05,^VIX,21.480000,23.090000,21.340000,22.750000,0
3,2012-01-06,^VIX,20.629999,21.719999,20.580000,21.240000,0
4,2012-01-09,^VIX,21.070000,21.780001,21.000000,21.670000,0
...,...,...,...,...,...,...,...
3511,2025-12-18,^VIX,16.870001,17.680000,15.930000,17.610001,0
3512,2025-12-19,^VIX,14.910000,16.530001,14.910000,16.309999,0
3513,2025-12-22,^VIX,14.080000,15.260000,14.030000,15.160000,0
3514,2025-12-23,^VIX,14.000000,14.450000,13.640000,14.090000,0


In [30]:
def calculate_sma(df, lookback):
    """
    Calculates a simple moving average for a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with a 'close' column.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: The DataFrame with a new column for the moving average.
    """
    # Create a copy to avoid modifying the original DataFrame
    df_sma = df.copy()

    # Calculate the simple moving average
    df_sma['SMA'] = df_sma['Close'].rolling(window=lookback).mean()

    # Calculate the difference between the SMA and the Close price
    df_sma['SMA_sign'] = np.sign(df_sma['Close'] - df_sma['SMA'])

    return df_sma



In [31]:
def analyze_sma_changes(df, vix_df, lookback, vix_threshold):
    """
    Analyzes the impact of Simple Moving Average bearish sign changes on price differences.

    Args:
        df (pd.DataFrame): The input DataFrame with historical price data.
        lookback (int): The number of periods for the moving average.

    Returns:
        pd.DataFrame: A DataFrame containing the sum of Next_Close_Diff at SMA sign changes by year,
                      including the lookback value.
    """

    # 1. Calculate SMA and signs for every day
    df_with_sma = calculate_sma(df=df.copy(), lookback=lookback)

    # 2. Merge VIX data into the main dataframe based on Date
    # We use 'left' join to keep all price dates and pull in the corresponding VIX close
    vix_subset = vix_df[['Date', 'Close']].rename(columns={'Close': 'VIX_Close'})
    df_with_sma = pd.merge(df_with_sma, vix_subset, on='Date', how='left')

    # 3. Identify ALL sign changes (crosses up AND crosses down)
    previous_sign = df_with_sma['SMA_sign'].shift(1)
    all_crosses_mask = (df_with_sma['SMA_sign'] != previous_sign) & (~previous_sign.isna())

    # Create a DataFrame of every signal event
    all_signals = df_with_sma[all_crosses_mask].copy()

    # 4. Calculate Bearish Return: (Entry Price - Exit Price) / Entry Price
    # Because all_signals contains the NEXT signal (the Bullish cross),
    # shift(-1) now correctly points to the "Exit".
    all_signals.loc[:, 'Next_Close_Return'] = (
        (all_signals['Close'] - all_signals['Close'].shift(-1)) / all_signals['Close']
    )

    # 5. FILTER FOR BEARISH ENTRIES + VIX THRESHOLD
    # - SMA_sign == -1 (Price crossed below SMA)
    # - VIX_Close > vix_threshold (Volatility is high enough)
    bearish_trades = all_signals[
        (all_signals['SMA_sign'] == -1) &
        (all_signals['VIX_Close'] > vix_threshold)
    ].copy()

    # 6. Clean up and Aggregate
    bearish_trades['Date'] = pd.to_datetime(bearish_trades['Date'])
    bearish_trades['Year'] = bearish_trades['Date'].dt.year

    yearly_results = bearish_trades.groupby('Year')['Next_Close_Return'].sum().reset_index()
    yearly_results['Lookback'] = lookback
    yearly_results['VIX_Threshold'] = vix_threshold

    return yearly_results[['Year', 'Lookback', 'VIX_Threshold','Next_Close_Return']]

In [32]:
# Initialize an empty list to store the results from each lookback value
results_list = []

# Iterate through each value in the specified range
for lookback_value in range(lookback_range[0], lookback_range[1] + 1):
    # Calculate SMA changes for the current lookback value
    #df_sma_result = analyze_sma_changes(df=data.copy(), lookback=lookback_value)
    df_sma_result = analyze_sma_changes(df=data.copy(), vix_df=vix_data, lookback=lookback_value, vix_threshold=vix_limit)
     # Append the result to the list
    results_list.append(df_sma_result)

# Concatenate all the DataFrames in the list into a single DataFrame
all_sma_results = pd.concat(results_list, ignore_index=True)

# Print the resulting DataFrame
print("DataFrame with Simple Moving Average analysis for different lookback values:")
display(all_sma_results)

DataFrame with Simple Moving Average analysis for different lookback values:


,Year,Lookback,VIX_Threshold,Next_Close_Return
0,2012,3,0,1.741747
1,2013,3,0,0.216757
2,2014,3,0,0.395458
3,2015,3,0,0.228710
4,2016,3,0,1.075154
...,...,...,...,...
261,2021,21,0,0.201505
262,2022,21,0,-0.193160
263,2023,21,0,0.714460
264,2024,21,0,0.057626


In [33]:
# Pivot the DataFrame
pivoted_sma_results = all_sma_results.pivot(index='Year', columns='Lookback', values='Next_Close_Return')

pivoted_sma_results = pivoted_sma_results.fillna(0)

# Display the pivoted DataFrame
print("Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Return as values:")
display(pivoted_sma_results)

Pivoted DataFrame with Lookback as columns, Year as rows, and Next_Close_Return as values:


Lookback,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,
2012,1.741747,1.929949,2.267622,2.252976,2.289627,1.934401,1.837664,1.730750,1.804663,1.405704,1.341566,1.447009,1.431146,1.431146,1.523173,1.587548,1.601911,1.448964,1.377382
2013,0.216757,0.153885,0.299553,0.254565,0.405335,0.412995,0.525814,0.030029,0.046300,-0.010389,0.316179,0.433482,0.282059,0.284372,0.308145,0.279297,0.201631,0.299487,0.302102
2014,0.395458,0.534357,0.935753,0.706487,0.384098,0.282546,0.237408,0.305029,0.192713,0.430621,0.467255,-0.013365,0.215409,0.215409,0.076339,-0.058169,-0.058169,0.027553,0.024574
2015,0.228710,0.241716,-0.032483,-0.391060,0.232561,0.195622,0.435841,0.307130,0.274107,-0.109924,-0.023532,0.401389,0.014843,-0.104406,0.012734,-0.072448,-0.129008,-0.179472,-0.002440
2016,1.075154,1.210715,1.018655,1.224011,1.461827,1.617067,1.476589,1.310427,1.122286,0.796251,0.752653,0.751999,0.859085,1.059190,1.032603,1.007821,0.937017,1.113866,1.103824
2017,1.210178,1.348694,1.360306,1.533616,1.527557,1.314757,1.377987,1.253439,1.182418,1.204729,1.028474,1.005900,1.067162,1.193623,1.263302,1.389760,1.422393,1.040717,0.979636
2018,-0.460092,-0.133553,-0.022895,-0.507526,0.567600,0.690205,0.490175,0.273597,0.205216,0.032521,-0.000599,-0.116651,-0.159250,-0.104315,-0.108310,-0.430370,-0.445335,-0.346766,-0.269659
2019,0.688441,0.433312,0.325821,0.426269,0.208814,0.524777,0.706166,0.688407,0.810429,0.688745,0.811400,0.749763,0.903648,0.897500,0.862084,0.722789,0.696407,0.693469,0.754359
2020,0.641397,0.999356,0.969514,0.712596,0.846736,0.428102,0.830977,0.784702,0.824414,0.529930,0.285580,0.269799,0.042031,0.042031,0.324510,0.298697,0.374672,0.272537,0.354342


In [34]:
# Ensure 'Date' column in the initial 'data' DataFrame is in datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract the year from the 'Date' column
data['Year'] = data['Date'].dt.year

# Group by year and get the first and last close prices
yearly_price_change = data.groupby('Year')['Close'].agg(['first', 'last'])

# Calculate the difference between the last and first close price for each year
yearly_price_change['Yearly_Return'] = (yearly_price_change['first'] - yearly_price_change['last']) / yearly_price_change['first']

# Drop the 'first' and 'last' columns
yearly_price_change = yearly_price_change.drop(columns=['first', 'last'])

# Display the result
print("Difference between the last and first close price of each year:")
display(yearly_price_change)

Difference between the last and first close price of each year:


,Yearly_Return
Year,
2012,0.967984
2013,0.895582
2014,0.639478
2015,0.766954
2016,0.945244
2017,0.931933
2018,-0.729735
2019,0.834574
2020,0.122012


In [35]:
# Join the yearly_price_change DataFrame with the yearly_next_close_diff_sum Series on the 'Year' index
comparison_df = yearly_price_change.join(pivoted_sma_results)

# Display the combined DataFrame, excluding the 'first' and 'last' columns
display(comparison_df)

,Yearly_Return,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Year,,,,,,,,,,,,,,,,,,,,
2012,0.967984,1.741747,1.929949,2.267622,2.252976,2.289627,1.934401,1.837664,1.730750,1.804663,1.405704,1.341566,1.447009,1.431146,1.431146,1.523173,1.587548,1.601911,1.448964,1.377382
2013,0.895582,0.216757,0.153885,0.299553,0.254565,0.405335,0.412995,0.525814,0.030029,0.046300,-0.010389,0.316179,0.433482,0.282059,0.284372,0.308145,0.279297,0.201631,0.299487,0.302102
2014,0.639478,0.395458,0.534357,0.935753,0.706487,0.384098,0.282546,0.237408,0.305029,0.192713,0.430621,0.467255,-0.013365,0.215409,0.215409,0.076339,-0.058169,-0.058169,0.027553,0.024574
2015,0.766954,0.228710,0.241716,-0.032483,-0.391060,0.232561,0.195622,0.435841,0.307130,0.274107,-0.109924,-0.023532,0.401389,0.014843,-0.104406,0.012734,-0.072448,-0.129008,-0.179472,-0.002440
2016,0.945244,1.075154,1.210715,1.018655,1.224011,1.461827,1.617067,1.476589,1.310427,1.122286,0.796251,0.752653,0.751999,0.859085,1.059190,1.032603,1.007821,0.937017,1.113866,1.103824
2017,0.931933,1.210178,1.348694,1.360306,1.533616,1.527557,1.314757,1.377987,1.253439,1.182418,1.204729,1.028474,1.005900,1.067162,1.193623,1.263302,1.389760,1.422393,1.040717,0.979636
2018,-0.729735,-0.460092,-0.133553,-0.022895,-0.507526,0.567600,0.690205,0.490175,0.273597,0.205216,0.032521,-0.000599,-0.116651,-0.159250,-0.104315,-0.108310,-0.430370,-0.445335,-0.346766,-0.269659
2019,0.834574,0.688441,0.433312,0.325821,0.426269,0.208814,0.524777,0.706166,0.688407,0.810429,0.688745,0.811400,0.749763,0.903648,0.897500,0.862084,0.722789,0.696407,0.693469,0.754359
2020,0.122012,0.641397,0.999356,0.969514,0.712596,0.846736,0.428102,0.830977,0.784702,0.824414,0.529930,0.285580,0.269799,0.042031,0.042031,0.324510,0.298697,0.374672,0.272537,0.354342


In [36]:
# Calculate the sum and standard deviation of each column in the comparison_df
grand_totals = comparison_df.sum()
standard_deviations = comparison_df.std()

# Calculate the Mean Absolute Deviation for each column
mean_absolute_deviations = comparison_df.apply(lambda x: (x - x.mean()).abs().mean())

# Combine the metrics into a single DataFrame for display
summary_df = pd.DataFrame({
    'Grand Total': grand_totals,
    'Standard Deviation': standard_deviations,
    'Mean Absolute Deviation': mean_absolute_deviations
})

# Display the summary DataFrame
print("Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:")
display(summary_df)

Grand Totals, Standard Deviations, and Mean Absolute Deviations for each column:


,Grand Total,Standard Deviation,Mean Absolute Deviation
Yearly_Return,8.729193,0.458569,0.310603
3,7.612810,0.618844,0.475456
4,8.275580,0.609901,0.446323
5,8.709609,0.660984,0.507795
6,8.211084,0.744152,0.544807
7,9.858806,0.660379,0.476953
8,8.805261,0.602678,0.434377
9,8.681899,0.598400,0.451615
10,7.113349,0.582800,0.473160
11,7.063944,0.580819,0.491829
